In [1]:
import numpy as np
from fractions import *

from time import *

import sys

sys.path.insert(1, "/Users/minhpham/Documents/Research/laughing-umbrella/xx_synthesis/monodromy")

import monodromy

from monodromy.coordinates import monodromy_alcove, monodromy_alcove_c2, monodromy_to_positive_canonical_polytope, rho_reflect
from monodromy.elimination import cylinderize, project
from monodromy.polytopes import ConvexPolytope, Polytope, make_convex_polytope
from monodromy.static import qlr_polytope
from monodromy.static.examples import empty_polytope

/Users/minhpham/Documents/Research/laughing-umbrella/xx_synthesis/monodromy/lrslib-071b/lrs


### Strength Polytope

In [2]:
strength_polytope = make_convex_polytope([
    # k s+ s1 s2 beta
    [999, -1,  0,  0,  0],  # 999 - s+ ≥ 0
    [  0,  1, -1, -1,  0],  # s+ - s1 - s2 ≥ 0
    [  0,  0,  1, -1,  0],  # s1 - s2 ≥ 0
    [  0,  0,  0,  1,  0],  # s2 ≥ 0
    [  1,  0, -4,  0,  0],  # 1/4 - s1 ≥ 0
    [  1,  0,  0,  0, -4],  # 1/4 - beta ≥ 0
    [  0,  0,  0,  0,  1],  # beta ≥ 0
], name="I")

First, we need to change ```strength_polytope``` to make sure that $\frac{\pi}{8} \geq \alpha^\prime \geq \alpha^{\prime\prime} \geq 0$. Furthermore, we need to change ```a_polytope``` and ```b_polytope``` to accomodate for the extra set of inequalities. 

In [3]:
a_polytope = make_convex_polytope([
    # k a1  a2  a3 s+ s1 s2
    [0,  1, -1,  0, 0, 0, 0],  # a1 - a2 ≥ 0
    [0,  0,  1, -1, 0, 0, 0],  # a2 - a3 ≥ 0
    [0,  0,  0,  1, 0, 0, 0],  # a3 ≥ 0
    [1, -1, -1,  0, 0, 0, 0],  # 1 - a1 - a2 ≥ 0
], name="A alcove").intersect(
    # MGC inequalities
    empty_polytope.union(make_convex_polytope([
        # strength
        [0, -1, -1, -1, 2,  0,  0],  # - a1 - a2 - a3 + 2s+ ≥ 0
        # slant
        [0,  1, -1, -1, 2, -2,  -2],  # a1 - a2 - a3 + 2s+ - 2s1 -2s2 ≥ 0
        # frustrum
        [0,  0,  0, -1, 1, -1, 0],  # - a3 + s+ - s1 ≥ 0
        # spade
        [0, -1, 0, 0, 1, 0, 0],  # - a1 + s+ ≥ 0
        # club
        [0, -1, 1, -1, 2, -2, 0], # -a1 + a2 - a3 + 2s+ - 2s1 ≥ 0
        # diamond
        [0, 0, 1, 0, 1, -2, 0], # a2 + s+ - 2s1 ≥ 0
        # heart
        [0, 1, 1, -1, 2, -4, 0], # a1 + a2 - a3 + 2s+ - 4s1 ≥ 0
    ], name="A unreflected")).union(make_convex_polytope([
        # strength
        [-1 , 1, -1, -1, 2,  0,  0],  # -1 + a1 - a2 - a3 + 2s+ ≥ 0
        # slant
        [1, -1, -1, -1, 2, -2,  -2],  # 1 - a1 - a2 - a3 + 2s+ - 2s1 - 2s2 ≥ 0
        # frustrum
        [0,  0,  0, -1, 1, -1, -0],  # - a3 + s+ - s1 ≥ 0
        # spade
        [-1, 1, 0, 0, 1, 0, 0],  # -1 + a1 + s+ ≥ 0
        # club
        [-1, 1, 1, -1, 2, -2, 0], # -1 + a1 + a2 - a3 + 2s+ - 2s1 ≥ 0
        # diamond
        [0, 0, 1, 0, 1, -2, 0], # a2 + s+ - 2s1 ≥ 0
        # heart
        [1, -1, 1, -1, 2, -4, 0], # 1 - a1 + a2 - a3 + 2s+ - 4s1 ≥ 0
    ], name="A reflected"))
)

In [4]:
b_polytope = make_convex_polytope([
    # k b1 b2 b3 s+ s1 s2 beta
    [0,  1, -1,  0, 0, 0, 0, 0],  # b1 - b2 ≥ 0
    [0,  0,  1, -1, 0, 0, 0, 0],  # b2 - b3 ≥ 0
    [0,  0,  0,  1, 0, 0, 0, 0],  # b3 ≥ 0
    [1, -1, -1,  0, 0, 0, 0, 0],  # 1 - b1 - b2 ≥ 0
], name="B alcove").intersect(
    # MGC inequalities
    empty_polytope.union(make_convex_polytope([
        # strength
        [0, -1, -1, -1, 2,  0,  0,  2],  # - b1 - b2 - b3 + 2s+ + 2beta ≥ 0
        # slant
        [0,  1, -1, -1, 2, -2,  -2,  2],  # b1 - b2 - b3 + 2s+ - 2s1 - 2s2 + 2beta ≥ 0
        [0,  1, -1, -1, 2,  -2,  0, 0],  # b1 - b2 - b3 + 2s+ - 2s1 ≥ 0
        # frustrum
        [0,  0,  0, -1, 1, -1, 0,  1],  # - b3 + s+ - s1 + beta ≥ 0
        [0,  0,  0, -1, 1, 0,  0,  0],  # - b3 + s+ ≥ 0
        # spade
        [0, -1, 0, 0, 1, 0, 0, 1], # - b1 + s+ + beta ≥ 0
        # club
        [0, -1, 1, -1, 2, -2, 0, 2], # - b1 + b2 - b3 + 2s+ -2s1 + 2beta ≥ 0
        [0, -1, 1, -1, 2, 0, 0, 0], # - b1 + b2 - b3 + 2s+ ≥ 0
        # diamond
        [0, 0, 1, 0, 1, -2, 0, 1], # b2 + s+ - 2s1 + beta ≥ 0
        [0, 0, 1, 0, 1, 0, 0, -1], # b2 + s+ - beta ≥ 0
        # heart
        [0, 1, 1, -1, 2, -4, 0, 2], # b1 + b2 - b3 + 2s+ - 4s1 + 2beta ≥ 0
        [0, 1, 1, -1, 2, 0, 0, -2], # b1 + b2 - b3 + 2s+ - 2beta ≥ 0
    ], name="B unreflected")).union(make_convex_polytope([
        # strength
        [-1 , 1, -1, -1, 2,  0,  0,  2],  # -1 + b1 - b2 - b3 + 2s+ + 2beta ≥ 0
        # slant
        [ 1, -1, -1, -1, 2,  -2,  -2, 2],  # 1 - b1 - b2 - b3 + 2s+ - 2s1 - 2s2 + 2beta ≥ 0
        [ 1, -1, -1, -1, 2, -2,  0,  0],  # 1 - b1 - b2 - b3 + 2s+ - 2s1 ≥ 0
        # frustrum
        [ 0,  0,  0, -1, 1, -1, 0,  1],  # - b3 + s+ - s1 + beta ≥ 0
        [ 0,  0,  0, -1, 1, 0,  0,  0],  # - b3 + s+ ≥ 0
        # spade
        [-1, 1, 0, 0, 1, 0, 0, 1],  # -1 + b1 + s+ + beta ≥ 0
        # club
        [-1, 1, 1, -1, 2, -2, 0, 2], # -1 + b1 + b2 - b3 + 2s+ - 2s1 + 2beta ≥ 0
        [-1, 1, 1, -1, 2, 0, 0, 0], # -1 + b1 + b2 - b3 + 2s+ ≥ 0
        # diamond
        [0, 0, 1, 0, 1, -2, 0, 1], # b2 + s+ - 2s1 + beta ≥ 0
        [0, 0, 1, 0, 1, 0, 0, -1], # b2 + s+ - beta ≥ 0
        # heart
        [1, -1, 1, -1, 2, -4, 0, 2], # 1 - b1 + b2 - b3 + 2s+ - 4s1 + 2beta ≥ 0
        [1, -1, 1, -1, 2, 0, 0, -2], # 1 - b1 + b2 - b3 + 2s+ - 2beta ≥ 0
    ], name="B reflected"))
)

In [5]:
def check_main_xy_theorem():
    """
    Checks the inductive step in the presentation of XY circuit polytopes.
    """
    global a_polytope, b_polytope, strength_polytope    
    
    print(f"Start running at {strftime('%Y-%m-%d %H:%M:%S', gmtime())}")
    start_time = perf_counter()

    biswas_relations = (qlr_polytope
        # enlarge to the pu_4 version of the QLR relations
        .union(rho_reflect(qlr_polytope, [0, 7, 8, 9]))
        # constrain in- and out-coordinates to the appropriate alcove
        .intersect(cylinderize(monodromy_alcove, [0, 1, 2, 3], 10))
        .intersect(cylinderize(monodromy_alcove_c2, [0, 7, 8, 9], 10))
    )

    # constrain interaction coordinates to be of XX-type
    biswas_relations = biswas_relations.intersect(Polytope(convex_subpolytopes=[
        ConvexPolytope(
            inequalities=[[1, 0, 0, 0, -4, 0, 0, 0, 0, 0]],
            equalities=[
                [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],  # x2 == 0
                [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],  # x3 == 0
            ]
        )
    ]))

    # switch to canonical coordinates
    biswas_relations = monodromy_to_positive_canonical_polytope(
        biswas_relations, coordinates=[0, 1, 2, 3])
    biswas_relations = monodromy_to_positive_canonical_polytope(
        biswas_relations, coordinates=[0, 4, 5, 6])
    biswas_relations = monodromy_to_positive_canonical_polytope(
        biswas_relations, coordinates=[0, 7, 8, 9])

    # reduce the biswas relations to have following coordinates:
    # k a1 a2 a3 beta b1 b2 b3
    biswas_relations = biswas_relations.reduce()
    biswas_relations = project(biswas_relations, 6).reduce()
    biswas_relations = project(biswas_relations, 5).reduce()

    # build the monodromy polytope relating the in-coordinates, constrained by
    # the inductive hypothesis, to the output coordinates through the biswas
    # relations + the XX interaction constraint. we'll get these coordinates:
    # k a1 a2 a3 b1 b2 b3 s+ s1 s2 beta
    induction_polytope = (empty_polytope
        # k a1 a2 a3 beta b1 b2 b3
        .union(cylinderize(biswas_relations, [0, 1, 2, 3, 10, 4, 5, 6], 11))
        # k a1 a2 a3 s+ s1 s2
        .intersect(cylinderize(a_polytope, [0, 1, 2, 3, 7, 8, 9], 11))
        # k s+ s1 s2 beta
        .intersect(cylinderize(strength_polytope, [0, 7, 8, 9, 10], 11)))

    # project away the a coordinates to get the next inductive step
    induction_polytope = project(induction_polytope, 3).reduce()
    induction_polytope = project(induction_polytope, 2).reduce()
    induction_polytope = project(induction_polytope, 1).reduce()

    # now check that this calculation actually closes the induction
    b_and_strength = b_polytope.intersect(
        cylinderize(strength_polytope, [0, 4, 5, 6, 7], 8)
    )

    print(induction_polytope.reduce())

    if b_and_strength.contains(induction_polytope) and \
        induction_polytope.contains(b_and_strength):
        stop_time = perf_counter()
        print(f"The statement is True.")
        print(f"The proof terminates in {round(stop_time - start_time, 3)} seconds.")
    else:
        stop_time = perf_counter()
        print(f"The statement is False.")
        print(f"The proof terminates in {round(stop_time - start_time, 3)} seconds.")

In [ ]:
check_main_xy_theorem()

Start running at 2024-07-07 21:37:40
